In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('/content/mastery_score_dataset-2.csv')
df2 = pd.read_csv('/content/mastery_score_dataset.csv')

In [7]:
df = pd.concat([df1, df2], ignore_index=True)

In [8]:
len(df)

300

In [9]:
# Define the features (inputs) and the target (output)
X = df[[
    "user_id",
    "concept",
    "difficulty_level",
    "quiz_score",
    "time_taken",
    "video_used",
    "article_used",
    "practice_problems_solved"
]]

y = df["mastery_score"]

In [10]:
from sklearn.preprocessing import LabelEncoder

# Encode the IDs and concept
user_encoder = LabelEncoder()
concept_encoder = LabelEncoder()

X["user_id"] = user_encoder.fit_transform(X["user_id"])
X["concept"] = concept_encoder.fit_transform(X["concept"])

# Store the encoders for future use
import pickle
with open("user_encoder.pkl", "wb") as f:
    pickle.dump(user_encoder, f)

with open("concept_encoder.pkl", "wb") as f:
    pickle.dump(concept_encoder, f)

<ipython-input-10-100562300>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["user_id"] = user_encoder.fit_transform(X["user_id"])
<ipython-input-10-100562300>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["concept"] = concept_encoder.fit_transform(X["concept"])


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

model1 = RandomForestRegressor(
    n_estimators=100,
    max_depth=5,
    random_state=42
)

model1.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, random_state=42)

In [16]:
predictions = model1.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

# Higher R^2 (closer to 1) means your model explains the variation well
# Mean Absolute Error (lower) means your predictions are close to the true values

Mean Absolute Error: 0.6641748234335151
R^2 Score: 0.7406618734751835


In [17]:
import xgboost as xgb

model2 = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)

model2.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [18]:
predictions = model2.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

# Higher R^2 (closer to 1) means your model explains the variation well
# Mean Absolute Error (lower) means your predictions are close to the true values

Mean Absolute Error: 0.7377723331451416
R^2 Score: 0.675877579722379


In [19]:
with open("xgboost_mastery_model.pkl", "wb") as f:
    pickle.dump(model2, f)

In [20]:
with open("randomforestregressor_mastery_model.pkl", "wb") as f:
    pickle.dump(model1, f)

In [23]:
# Assuming your combined DataFrame is called combined_df
df.to_csv("combined_data.csv", index=False)
print("Combined DataFrame successfully exported to combined_data.csv")

Combined DataFrame successfully exported to combined_data.csv


## Testing

In [24]:
import pickle
import xgboost as xgb
import pandas as pd

# Loading the trained model
with open("xgboost_mastery_model.pkl", "rb") as f:
    model = pickle.load(f)

# Loading the label encoders if applicable
with open("user_encoder.pkl", "rb") as f:
    user_encoder = pickle.load(f)

with open("concept_encoder.pkl", "rb") as f:
    concept_encoder = pickle.load(f)

In [30]:
# Provide custom input here
custom_input = {
    "user_id": [3693],
    "concept": ["Graphs"],
    "difficulty_level": [3],
    "quiz_score": [0.5],
    "time_taken": [7],
    "video_used": [1],
    "article_used": [1],
    "practice_problems_solved": [12],
}

input_df = pd.DataFrame(custom_input)

In [31]:
input_df["user_id"] = user_encoder.transform(input_df["user_id"])
input_df["concept"] = concept_encoder.transform(input_df["concept"])

# Now we’re ready for prediction

In [32]:
predicted_mastery = model.predict(input_df)[0]

print(f"Predicted Mastery Score: {predicted_mastery}")

Predicted Mastery Score: 4.2714080810546875
